In [ ]:
import os
import pandas as pd
import plotly.express as px
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())

#### Process crime data

In [ ]:
DATA_PATH = 'data'

In [ ]:
df_path_list = glob.glob(DATA_PATH + '/**/*.csv')

In [ ]:
df_month_list = []

In [ ]:
for df_path in df_path_list:
    df_month = pd.read_csv(df_path)
    df_month = df_month.drop(columns=['Reported by', 'Falls within', 'Last outcome category', 'Context'])
    df_month_list.append(df_month)

In [ ]:
df = pd.read_csv('data/filtered_data.csv')

In [ ]:
df = pd.concat(df_month_list).reset_index(drop=True)

In [ ]:
df = df[df['Crime ID'].notna()].reset_index(drop=True)

In [ ]:
# Filter crime data based on type
df = df[~df['Crime type'].isin(['Robbery', 'Bicycle theft', 'Shoplifting', 'Vehicle crime', 'Criminal damage and arson', 'Other theft'])].reset_index(drop=True)

In [ ]:
df

In [ ]:
# Slect only Manchester crimes based on osmnx graph coordinates
df = df[(53.4330 <= df['Latitude']) & (df['Latitude'] <= 53.5177) & (-2.3068 <= df['Longitude']) & (df['Longitude'] <= -2.1633)]

In [ ]:
df.to_csv('data/filtered_data.csv')

#### Load data

In [ ]:
df = pd.read_csv('data/filtered_data.csv')

In [ ]:
len(df.Location.unique())

In [ ]:
len(df.Longitude.unique())

#### Visualize stats

In [ ]:
plt.figure(figsize=(10, 5))
df['Crime type'].value_counts().plot.barh()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
df['Month'].value_counts().plot.barh()
plt.show()

#### Plot density map

In [ ]:
fig = px.density_mapbox(df, lat='Latitude', lon='Longitude', radius=5,
                        center=dict(lat=df.Latitude.mean(), lon=df.Longitude.mean()), zoom=10, height=600)

#### Model distribution

In [ ]:
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal

In [ ]:
# empirically determined tried elbow, silouhette
NUM_CLUSTERS = 25

In [ ]:
x = df.Longitude.values
y = df.Latitude.values

x_min = x.min()
y_min = y.min()
x_scale = 9518 / (x.max() - x.min())
y_scale = 9426 / (y.max() - y.min())

x = (x - x.min()) / (x.max() - x.min()) * 9518
y = (y - y.min()) / (y.max() - y.min()) * 9426

In [ ]:
points = np.stack([x, y]).T

In [ ]:
points.shape

In [ ]:
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=0).fit(points)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(points[:, 0], points[:, 1], c=kmeans.labels_)

for i, c in enumerate(kmeans.cluster_centers_):
    draw_circle = plt.Circle(c, 300, color='white')
    plt.gcf().gca().add_artist(draw_circle)
    draw_circle = plt.Circle(c, 300, fill=False)
    plt.gcf().gca().add_artist(draw_circle)
    plt.text(c[0] - 125, c[1] - 50, str(i), fontsize=15)

# plt.savefig('images/normalized_clustering_numbered.jpg', bbox_inches='tight')
plt.show()

#### Model bivariate normal distributions

In [ ]:
distr = []
X, Y = np.meshgrid(np.linspace(0, x.max(), num=100), np.linspace(0, y.max(), num=100))
max_val = 0

In [ ]:
for c in range(NUM_CLUSTERS):
    cov = np.cov(points[kmeans.labels_ == c].T)
    mean = kmeans.cluster_centers_[c]
    distr.append(multivariate_normal(cov=cov, mean=mean, seed=0))

    if max_val < distr[-1].pdf(mean):
        max_val = distr[-1].pdf(mean)

In [ ]:
# Generating the density function for each point in the meshgrid
pdf = np.zeros(X.shape)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        pdf[i,j] = sum([distr[t].pdf([X[i,j], Y[i,j]]) for t in range(NUM_CLUSTERS)])

In [ ]:
fig = plt.figure(figsize=(20, 20))

cmap = plt.get_cmap('jet')
pp = plt.contourf(X, Y, pdf,cmap=cmap, alpha=.3, antialiased=True, levels=50)
plt.scatter(points[:, 0], points[:, 1], c=kmeans.labels_,)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, pdf / pdf.max(), cmap='viridis')

#### Precompute pdf

In [ ]:
precalc = np.zeros((1000, 1000))

In [ ]:
for i in range(1000):
    if i % 10 == 0:
        print(i)
    for j in range(1000):
        precalc[i, j] = sum([distr[t].pdf([i * 10, j * 10]) for t in range(NUM_CLUSTERS)])

#### Save distribution

In [ ]:
import pickle

In [ ]:
with open('Inference/distribution_k25_precalc.obj', 'wb') as f:
    pickle.dump((distr, max_val, x_min, x_scale, y_min, y_scale, precalc), f)